# Imports


In [1]:
import os 
import sys

In [6]:
# Set path to parent dir to import personal imports
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from data.TrajectorySet import TrajectorySet
from utils.truncated_distributions import truncated_normal
from utils.truncated_distributions import truncated_laplace
from utils.truncated_distributions import truncated_exponential

import torch 
import minari 
import numpy as np 


In [3]:
dataset = minari.load_dataset("D4RL/pointmaze/large-v2")

# Sampler

### Need to sample and get positive and negative pairs (states). 

In [7]:
class Sampler(): 
    def __init__(self, T: TrajectorySet, dist="g", sigma = 15, b = 15, rate = 0.99): 
        """
        T: The Trajectory Set class 
        dist: The distribution used for centering over the anchor state. 
            ['u', 'g', 'l', 'e'] - uniform, gaussian, laplace, exponential
        """

        self.T = T 
        self.dist = dist
        
        # Hyperparameters
        self.sigma = sigma
        self.b = b 
        self.rate = rate  


    def sample_anchor_state(self, t: list) -> tuple[list, int]: 
        """
        Given a trajectory, we sample the anchor state s_i uniformly. 

        Args: 
            t: The given trajectory we sample from. 

        Returns: 
            A tuple containing [s_i, idx]
            s_i: The state that is sampled, represented as a list of (x,y) coordinates and velocities. 
            idx: The time step of s_i. 
        """
        idx = torch.randint(low=0, high=len(t), size=(1,)).item()
        s_i = t[idx] 
        return [s_i, idx]


    def sample_positive_pair(self, t: list, anchor_state: tuple[list, int]) -> tuple[list, int]: 
        """
        Given the same trajectory that s_i was sampled from, 
        center a gaussian distribution around s_i to get obtain its positve pair: s_j. 
        
        Args: 
            t: The given trajectory, which must be the same as the trajectory that was used to sample the anchor state. 
            anchor_state: The anchor state; a tuple containing [s_i, idx].
            s_i: The state itself.
            idx: The time step of s_i.
            
        Return: 
            Returns the positive pair's state and state index.
        """

        _, si_idx = anchor_state

        if self.dist == "u": 
            # uniform 
            sj_idx = torch.randint(low=0, high=len(t), size=(1,))

        elif self.dist == "g": 
            # gaussian 
            p = truncated_normal(len(t), mu=si_idx, sigma=self.sigma) 
            sj_idx = np.random.choice(a=len(t), p=p)
            
        elif self.dist == "l": 
            # laplacian
            p = truncated_laplace(len=len(t), mu=si_idx, b=self.b)
            sj_idx = np.random.choice(a=len(t), p=p)

        elif self.dist == "e": 
            # exponential 
            p = truncated_exponential(len=len(t), anchor_state_index=si_idx, rate=self.rate)
            sj_idx = np.random.choice(a=len(t), p=p) 

        else: 
            # default to gaussian
            p = truncated_normal(len(t), mu=si_idx, sigma=self.sigma) 
            sj_idx = np.random.choice(a=len(t), p=p)
        
        s_j = t[sj_idx]
        return [s_j, sj_idx]
    

    def sample_batch(self, batch_size=1024, k=2) -> list[tuple]: 
        """ 
        Creates a batch of anchor states, their positive pairs, and negative pairs. 
        There will be 2(batch_size - 1) amount of negative examples per positive pair.

        Args: 
            batch_size: The size of the batch to be generated.
            k: A hyperparameter that dictates the average number of 
                positive pairs sampled from the same trajectory. The 
                lower the number, the lesser the chance of false negatives. 
        
        Returns: 
            A list of tuples containing the anchor_state and its positive pair. 
            The list is the same length as batch_size. 
        """ 

        batch = [] 

        # Generate trajectory set 
        n_trajectories = batch_size // k
        self.T.generate_trajectories(n_trajectories= n_trajectories)

        for _ in range(batch_size): 
            # Sample anchor state 
            rng = torch.randint(low=0, high=n_trajectories, size=(1,)).item() 
            t = self.T.get_trajectory(index=rng)[0]
            
            anchor_state = self.sample_anchor_state(t) 

            # Sample positive pair 
            positive_pair = self.sample_positive_pair(t, anchor_state=anchor_state)

            # Retrieve states; time-steps aren't necessary. 
            s_i = anchor_state[0]
            s_j = positive_pair[0]

            batch.append([s_i, s_j]) 

        return batch 

In [14]:
"""
TESTING CELL!
"""
T = TrajectorySet(dataset=dataset)

uniform = "u"
gaussian = "g"
laplacian = "l"
exponential = "e"

s = Sampler(T, laplacian)

T.generate_trajectories(n_trajectories=1) 
t = T.get_trajectory(0)[0] 
anchor_state = s.sample_anchor_state(t)
print(f"s_i: {anchor_state[0]}, at time step: {anchor_state[1]}")

positive_pair = s.sample_positive_pair(t=t, anchor_state=anchor_state)
print(f"s_j: {positive_pair[0]}, at time step: {positive_pair[1]}")


s_i: [-0.14469304  0.98291039 -4.35667975 -0.5873383 ], at time step: 157
s_j: [-0.33059188  0.9611728  -3.22394094 -0.22612503], at time step: 162


In [15]:
"""
TESTING CELL! 

Testing to see if sample_batch works properly. 
"""

batch = s.sample_batch(batch_size=128, k=2)
print("Batch:", batch)
len(batch)

Batch: [[array([ 0.56870167,  2.41734414, -2.1953144 , -2.82081276]), array([ 0.73601277,  2.5516822 , -3.48755141, -1.42048912])], [array([-4.16956237,  2.9079329 , -4.81422535,  0.06935036]), array([-3.36274968,  2.84272199, -3.87854267,  0.11520598])], [array([-3.50391923e+00, -2.70694707e+00, -1.58601976e-03,  3.86680992e+00]), array([-3.56736572, -1.84723318, -1.09876067,  4.06927189])], [array([ 0.45081867,  2.44274512, -0.01286939,  4.71943654]), array([0.45267535, 2.78909559, 0.53293338, 2.39661628])], [array([-1.012238  ,  0.89482413, -5.03012776,  1.13837793]), array([ 0.44761699,  0.5602349 , -0.19576949,  4.7233684 ])], [array([ 4.45895717,  3.00595937, -1.31085017,  1.37946242]), array([ 4.49120896,  2.95730895, -0.60235248,  2.1072741 ])], [array([ 0.7489631 ,  2.93579248,  4.21883557, -0.105629  ]), array([ 0.70677474,  2.93684877,  3.99017488, -0.05372305])], [array([ 2.47168207, -1.57292286, -0.17360297,  4.88771031]), array([ 2.46703662, -1.48244743, -0.18362607,  4.4

128